In [1]:
! pip install psutil

In [2]:
! pip install keras==2.1.4

In [0]:
!pip install -U -q PyDrive

In [4]:
import keras
import psutil
import pandas as pd
import tensorflow as tf
from google.colab import auth
from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

Using TensorFlow backend.


In [5]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file1 = drive.CreateFile({'id':'1Pqjs7c5MAgPQLiMA3YR8dFg_YHLnPdlO'})
file1.GetContentFile('dataset.zip')

In [8]:
!unzip dataset.zip -dq ./ 

Archive:  dataset.zip
caution: filename not matched:  ./


In [0]:
file2 = drive.CreateFile({'id':'18H1VBB5It4tneouiOhTqSQjFscOomAum'})
file2.GetContentFile('ernest.py')

In [0]:
from ernest import *

In [12]:
clean_train("dataset//train")

computing md5 of training data
Files appearing more than once in train:  33
0 images deleted from training set


In [13]:
_ = find_leak("dataset//train", "dataset//test")

computing md5 of training data
comparing training and test set
Number of test images present in train:0


In [0]:
process_train_images("dataset//train")

In [0]:
process_test_images("dataset//test//data")

In [0]:
create_val_set(0.2)

In [18]:
model = define_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
train_datagen = ImageDataGenerator(
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        preprocessing_function = preprocess_input
)

test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [20]:
train_generator = train_datagen.flow_from_directory(
        "dataset//train",
        target_size = TARGET_SIZE, 
        batch_size = BATCH_SIZE, 
        class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
        "dataset//valid", 
        target_size = TARGET_SIZE, 
        batch_size = BATCH_SIZE, 
        class_mode = "categorical")

test_generator = test_datagen.flow_from_directory(
        "dataset//test", 
        target_size = TARGET_SIZE, 
        batch_size = BATCH_SIZE, 
        class_mode = None, 
        shuffle = False)

Found 4350 images belonging to 12 classes.
Found 1083 images belonging to 12 classes.
Found 3375 images belonging to 1 classes.


In [0]:
earlystop = EarlyStopping(monitor = "val_loss", 
                          patience = 10, 
                          verbose = 1, 
                          mode = "auto")

checkpoint = ModelCheckpoint(filepath = "inceptionv3.hdf5", 
                             verbose = 1, 
                             save_best_only = True)

reducelr = ReduceLROnPlateau(monitor = "val_loss", 
                             factor = 0.2,
                             patience = 2)


In [22]:
model.fit_generator(
        generator = train_generator,
        epochs = 10,
        callbacks = [checkpoint, earlystop, reducelr],
        validation_data = validation_generator,
        verbose = 1)

Epoch 1/10
136/136 [==============================] - 137s 1s/step - loss: 2.2600 - acc: 0.2414 - val_loss: 2.2270 - val_acc: 0.2493

Epoch 00001: val_loss improved from inf to 2.22704, saving model to inceptionv3.hdf5
Epoch 2/10
136/136 [==============================] - 125s 923ms/step - loss: 1.8726 - acc: 0.3722 - val_loss: 2.1949 - val_acc: 0.2687

Epoch 00002: val_loss improved from 2.22704 to 2.19493, saving model to inceptionv3.hdf5
Epoch 3/10
 81/136 [================>.............] - ETA: 41s - loss: 1.6693 - acc: 0.4568

136/136 [==============================] - 125s 919ms/step - loss: 1.6774 - acc: 0.4513 - val_loss: 2.1773 - val_acc: 0.2724

Epoch 00003: val_loss improved from 2.19493 to 2.17725, saving model to inceptionv3.hdf5
Epoch 4/10
136/136 [==============================] - 127s 937ms/step - loss: 1.5514 - acc: 0.4847 - val_loss: 2.0518 - val_acc: 0.3176

Epoch 00004: val_loss improved from 2.17725 to 2.05176, saving model to inceptionv3.hdf5
Epoch 5/10
 98/136 [====================>.........] - ETA: 30s - loss: 1.4563 - acc: 0.5191

136/136 [==============================] - 127s 930ms/step - loss: 1.4435 - acc: 0.5223 - val_loss: 2.0418 - val_acc: 0.3352

Epoch 00005: val_loss improved from 2.05176 to 2.04183, saving model to inceptionv3.hdf5
Epoch 6/10
136/136 [==============================] - 128s 939ms/step - loss: 1.4167 - acc: 0.5304 - val_loss: 1.9870 - val_acc: 0.3601

Epoch 00006: val_loss improved from 2.04183 to 1.98702, saving model to inceptionv3.hdf5
Epoch 7/10
101/136 [=====================>........] - ETA: 28s - loss: 1.4038 - acc: 0.5337

136/136 [==============================] - 128s 942ms/step - loss: 1.3967 - acc: 0.5389 - val_loss: 2.0003 - val_acc: 0.3250

Epoch 00007: val_loss did not improve
Epoch 8/10
136/136 [==============================] - 123s 902ms/step - loss: 1.3073 - acc: 0.5619 - val_loss: 2.0705 - val_acc: 0.3352

Epoch 00008: val_loss did not improve
Epoch 9/10
136/136 [==============================] - 124s 911ms/step - loss: 1.2577 - acc: 0.5816 - val_loss: 1.9433 - val_acc: 0.3878

Epoch 00009: val_loss improved from 1.98702 to 1.94331, saving model to inceptionv3.hdf5
Epoch 10/10
  3/136 [..............................] - ETA: 1:01 - loss: 1.1867 - acc: 0.5833

136/136 [==============================] - 123s 901ms/step - loss: 1.2450 - acc: 0.5867 - val_loss: 1.9625 - val_acc: 0.3629

Epoch 00010: val_loss did not improve


In [23]:
predictions = model.predict_generator(
        test_generator,
        verbose = 1)

106/106 [==============================] - 51s 477ms/step


In [0]:
predictions = np.argmax(predictions, axis = 1)
labels = train_generator.class_indices
predictions = [list(labels.keys())[list(labels.values()).index(i)] for i in predictions]

In [0]:
json_model = model.to_json()
with open("inceptionV3.json", "w") as json_file:
    json_file.write(json_model)
    
model.save_weights("inceptionV3.h5")

In [0]:
new_subm = pd.DataFrame({"filename": test_generator.filenames, "Superhero": predictions})
new_subm["filename"] = new_subm["filename"].apply(lambda x: x.split("/")[1])
new_subm["filename"] = new_subm["filename"].apply(lambda x: x.split(".")[0])

In [0]:
subm = pd.read_csv("dataset//Superhero_Submission_Format.csv")
del subm["Superhero"]
subm = subm.join(new_subm.set_index("filename"), on = "filename")

In [0]:
subm.loc[subm["Superhero"] == "Spiderman", "Superhero"] = "spider_man"
subm.loc[subm["Superhero"] == "Ghost Rider", "Superhero"] = "ghostrider"
subm.loc[subm["Superhero"] == "Captain America", "Superhero"] = "captain_america"
subm.loc[subm["Superhero"] == "Ant-Man", "Superhero"] = "ant_man"
subm.loc[subm["Superhero"] == "Hulk", "Superhero"] = "hulk"
subm.loc[subm["Superhero"] == "Catwoman", "Superhero"] = "cat_woman"
subm.loc[subm["Superhero"] == "Iron Man", "Superhero"] = "iron_man"
subm.loc[subm["Superhero"] == "Black Panther", "Superhero"] = "black_panther"
subm.loc[subm["Superhero"] == "Batman", "Superhero"] = "bat_man"
subm.loc[subm["Superhero"] == "Avengers", "Superhero"] = "avengers"
subm.loc[subm["Superhero"] == "Superman", "Superhero"] = "super_man"
subm.loc[subm["Superhero"] == "Aquaman", "Superhero"] = "aqua_man"

In [0]:
subm.to_csv("submission_0323_a.csv", index = False)
files.download("submission_0323.csv")

In [34]:
file3 = drive.CreateFile({"title": "inceptionV3"})
file3.Upload()
print('title: %s, id: %s' % (file3['title'], file3['id']))

title: inceptionV3, id: 193czxV0V5nJa9qJtv7c-_UAUkUfcdTEf


In [0]:
file3 = drive.CreateFile({"id": "193czxV0V5nJa9qJtv7c-_UAUkUfcdTEf"})
file3.SetContentFile('./inceptionV3.h5')

In [0]:
file3.Upload()